<a href="https://colab.research.google.com/github/akram01Br/PixelHawk-OD/blob/main/Brain_Tumor_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This project involves building a deep learning model to classify brain tumors from MRI scans into four
#categories: glioma, meningioma, pituitary tumor, and no tumor. The solution includes data preprocessing,
#model building, evaluation, and deployment.
# Brain Tumor Classification
# Author: [Your Name]
# Date: [Current Date]

## 1. Project Overview
"""
This project aims to classify brain tumors from MRI scans using deep learning.
The dataset contains MRI images categorized into 4 classes:
- Glioma
- Meningioma
- Pituitary tumor
- No tumor

Skills demonstrated:
- Medical image processing
- Deep learning with TensorFlow/Keras
- Data augmentation
- Model evaluation
- Gradio app deployment
"""

## 2. Setup and Installation
!pip install tensorflow keras opencv-python matplotlib numpy pandas scikit-learn gradio













In [ ]:
# 1. Install Kaggle API (if not already installed)
!pip install -q kaggle

# 2. Upload your Kaggle API key
from google.colab import files
files.upload()  # Upload your kaggle.json file here

# 3. Set up Kaggle API key
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 4. Download the dataset (using the correct identifier)
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

# 5. Unzip the dataset (correct filename)
!unzip -q brain-tumor-mri-dataset.zip -d brain_tumor_data

# 6. Verify the files
!ls brain_tumor_data/Data

Saving archive.zip to archive.zip
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open brain-tumor-mri-dataset.zip, brain-tumor-mri-dataset.zip.zip or brain-tumor-mri-dataset.zip.ZI

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sartajbhuvaji/brain-tumor-classification-mri")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/brain-tumor-classification-mri


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gradio as gr

In [ ]:
## 3. Data Loading and Exploration
# Download the dataset (using the Brain Tumor MRI Dataset from Kaggle)
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset
!unzip brain-tumor-mri-dataset.zip


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open brain-tumor-mri-dataset.zip, brain-tumor-mri-dataset.zip.zip or brain-tumor-mri-dataset.zip.ZIP.


In [ ]:
# Define paths
data_dir = '/content/Data'
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [ ]:
# Explore dataset
image_counts = {}
for cls in classes:
    image_counts[cls] = len(os.listdir(os.path.join(data_dir, cls)))

plt.figure(figsize=(10, 5))
plt.bar(image_counts.keys(), image_counts.values())
plt.title('Class Distribution')
plt.xlabel('Tumor Type')
plt.ylabel('Number of Images')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/Data/glioma'

In [ ]:
## 4. Data Preprocessing
# Image preprocessing function
def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize to [0,1]
    return image

# Load and preprocess all images
X = []
y = []
for i, cls in enumerate(classes):
    for image_name in os.listdir(os.path.join(data_dir, cls)):
        image_path = os.path.join(data_dir, cls, image_name)
        try:
            image = preprocess_image(image_path)
            X.append(image)
            y.append(i)
        except:
            pass

X = np.array(X)
y = np.array(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 5. Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)

## 6. Model Building
def create_model(input_shape=(224, 224, 3), num_classes=4):
    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )

    # Freeze base model layers
    base_model.trainable = False

    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

model = create_model()
model.summary()

## 7. Model Training
early_stopping = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3)

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

## 8. Model Evaluation
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()

# Classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_classes, target_names=classes))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

## 9. Model Deployment with Gradio
def predict_tumor(image):
    image = np.array(image)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)

    pred = model.predict(image)
    pred_class = classes[np.argmax(pred)]
    confidence = np.max(pred) * 100

    return {cls: float(pred[0][i]) for i, cls in enumerate(classes)}

# Create Gradio interface
iface = gr.Interface(
    fn=predict_tumor,
    inputs=gr.Image(label="Upload MRI Scan"),
    outputs=gr.Label(num_top_classes=4),
    title="Brain Tumor Classifier",
    description="Upload an MRI scan to classify the brain tumor type.",
    examples=[
        ["/content/Data/glioma/Tr-gl_0010.jpg"],
        ["/content/Data/meningioma/Tr-me_0010.jpg"],
        ["/content/Data/pituitary/Tr-pi_0010.jpg"],
        ["/content/Data/notumor/Tr-no_0010.jpg"]
    ]
)

iface.launch(debug=True)

FileNotFoundError: [Errno 2] No such file or directory: '/content/Data/glioma'